In [1]:
import saspy
sas = saspy.SASsession(colorLOG=True)

Using SAS Config named: oda
SAS Connection established. Subprocess id is 22500



### cars

In [2]:
sd = sas.sasdata("cars","sashelp")
sd.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945.0,33337.0,3.5,6.0,265.0,17.0,23.0,4451.0,106.0,189.0
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820.0,21761.0,2.0,4.0,200.0,24.0,31.0,2778.0,101.0,172.0
2,Acura,TSX 4dr,Sedan,Asia,Front,26990.0,24647.0,2.4,4.0,200.0,22.0,29.0,3230.0,105.0,183.0
3,Acura,TL 4dr,Sedan,Asia,Front,33195.0,30299.0,3.2,6.0,270.0,20.0,28.0,3575.0,108.0,186.0
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755.0,39014.0,3.5,6.0,225.0,18.0,24.0,3880.0,115.0,197.0


In [6]:
sas.submit(f"""
    proc freq data=sashelp.cars order=freq noprint;
        tables type * origin / out=_tmp;
    run;
""")
df = sas.sasdata("_tmp","work").to_df()
df

,Type,Origin,COUNT,PERCENT
0,Sedan,Asia,94.0,21.962617
1,Sedan,USA,90.0,21.028037
2,Sedan,Europe,78.0,18.224299
3,SUV,Asia,25.0,5.841121
4,SUV,USA,25.0,5.841121
5,SUV,Europe,10.0,2.336449
6,Sports,Asia,17.0,3.971963
7,Sports,USA,9.0,2.102804
8,Sports,Europe,23.0,5.373832
9,Wagon,Asia,11.0,2.570093


In [ ]:
_ = sd.where("1=1")
_.dsopts["keep"] = "msrp invoice"
_.means()

,Variable,N,NMiss,Median,Mean,StdDev,Min,P25,P50,P75,Max
0,MSRP,428.0,0.0,27635.0,32774.855140,19431.716674,10280.0,20329.5,27635.0,39215.0,192465.0
1,Invoice,428.0,0.0,25294.5,30014.700935,17642.117750,9875.0,18851.0,25294.5,35732.5,173560.0


In [7]:
sas.submitLST(f"""
    data cars;
        set sashelp.cars;
        keep type origin msrp invoice mpg_city mpg_highway;
    proc print data=cars (obs=5);
    run;
""")

Obs,Type,Origin,MSRP,Invoice,MPG_City,MPG_Highway
1,SUV,Asia,"$36,945","$33,337",17,23
2,Sedan,Asia,"$23,820","$21,761",24,31
3,Sedan,Asia,"$26,990","$24,647",22,29
4,Sedan,Asia,"$33,195","$30,299",20,28
5,Sedan,Asia,"$43,755","$39,014",18,24


In [ ]:
# 3 category prediction
sas.submitLST(f"""
    proc logistic data=sashelp.cars order=freq noprint;
        model origin (event='Asia') = mpg_city mpg_highway weight / link=glogit;
        ouput out=cars_est predprobs=INDIVIDUAL;
    
    proc print data=cars_est (obs=5);
        var origin mpg_city mpg_highway weight _from_ _into_;
    run;
""")

Obs,Origin,MPG_City,MPG_Highway,Weight,_FROM_,_INTO_
1,Asia,17,23,4451,Asia,USA
2,Asia,24,31,2778,Asia,Asia
3,Asia,22,29,3230,Asia,Asia
4,Asia,20,28,3575,Asia,USA
5,Asia,18,24,3880,Asia,USA


### heart

In [ ]:
sas.sasdata("heart","sashelp").head()

,Status,DeathCause,AgeCHDdiag,Sex,AgeAtStart,Height,Weight,Diastolic,Systolic,MRW,Smoking,AgeAtDeath,Cholesterol,Chol_Status,BP_Status,Weight_Status,Smoking_Status
0,Dead,Other,NaN,Female,29.0,62.50,140.0,78.0,124.0,121.0,0.0,55.0,NaN,NaN,Normal,Overweight,Non-smoker
1,Dead,Cancer,NaN,Female,41.0,59.75,194.0,92.0,144.0,183.0,0.0,57.0,181.0,Desirable,High,Overweight,Non-smoker
2,Alive,NaN,NaN,Female,57.0,62.25,132.0,90.0,170.0,114.0,10.0,NaN,250.0,High,High,Overweight,Moderate (6-15)
3,Alive,NaN,NaN,Female,39.0,65.75,158.0,80.0,128.0,123.0,0.0,NaN,242.0,High,Normal,Overweight,Non-smoker
4,Alive,NaN,NaN,Male,42.0,66.00,156.0,76.0,110.0,116.0,20.0,NaN,281.0,High,Optimal,Overweight,Heavy (16-25)


In [ ]:
sas.submitLST(f"""
data bmi_groups;
    set sashelp.heart;
    BMI = weight / (height**2) * 703;
    
    if BMI < 18.5 then bmi_range = "Underweight";
    else if BMI < 25 and BMI >= 18.5 then bmi_range = "Normal";
    else if BMI < 30 then bmi_range = "Overweight";
    else bmi_range = "Obese";
              
    if bmi_range = "Underweight" then bmi_range_n = "1. Underweight";
    else if bmi_range = "Normal" then bmi_range_n = "2. Normal";
    else if bmi_range = "Overweight" then bmi_range_n = "3. Overweight";
    else if bmi_range = "Obese" then bmi_range_n = "4. Obese";
    else bmi_range_n = "5. Other";
              
    keep weight height bmi bmi_range bmi_range_n;
run;

""" , method="listonly")
sas.sasdata('bmi_groups').head()


,Height,Weight,BMI,bmi_range,bmi_range_n
0,62.50,140.0,25.195520,Overweight,3. Overweight
1,59.75,194.0,38.201572,Obese,4. Obese
2,62.25,132.0,23.946969,Normal,2. Normal
3,65.75,158.0,25.693360,Overweight,3. Overweight
4,66.00,156.0,25.176309,Overweight,3. Overweight


### iris

In [ ]:
sas.sasdata("iris","sashelp").head()

,Species,SepalLength,SepalWidth,PetalLength,PetalWidth
0,Setosa,50.0,33.0,14.0,2.0
1,Setosa,46.0,34.0,14.0,3.0
2,Setosa,46.0,36.0,10.0,2.0
3,Setosa,51.0,33.0,17.0,5.0
4,Setosa,55.0,35.0,13.0,2.0


### orders

In [5]:
sas.submitLST(f"""
DATA orders;
    INPUT Coffee $ Window $ @@;
    datalines;
esp w cap d cap w kon w ice w kon d esp d kon w ice d esp d
cap w esp d cap d Kon d . d kon w esp d cap w ice w kon w
kon w kon w ice d esp d kon w esp d esp w kon w cap w kon w
;
""", method="listonly")
sas.sasdata('orders').head()

,Coffee,Window
0,esp,w
1,cap,d
2,cap,w
3,kon,w
4,ice,w


### prdsale

In [ ]:
sas.sasdata("prdsale","sashelp").head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,925.0,850.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-01-01
1,999.0,297.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-02-01
2,608.0,846.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-03-01
3,642.0,533.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-04-01
4,656.0,646.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-05-01


##